<a href="https://colab.research.google.com/github/Kirutkar/airesidency/blob/main/langchain_chains_agent_12thjul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain setup

In [1]:
!pip install openai langchain langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [2]:
#secret Key

import os
from google.colab import userdata

# Retrieve API keys from Colab's secure storage

openai_api_key = userdata.get("openai")

# Set them as environment variables

if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key

## Chains

Restaraunt Business Generator

In [3]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature = 0.9)

prompt_template_name= PromptTemplate(
    input_variables = ['cuisine'],
    template = "i want to open a restaraunt for  {cuisine} food. Suggest a fancy name for this"
                                         )
name_chain=LLMChain(llm=llm,prompt=prompt_template_name,output_key="restaraunt_name")

llm = OpenAI(temperature = 0.9)
prompt_template_items = PromptTemplate(input_variables=['restaraunt_name'],
                                      template="Suggest me some menu items for {restaraunt_name}."
                                       )

food_items_chain=LLMChain(llm=llm,prompt=prompt_template_items,output_key="menu_items")



from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables =['cuisine',],
    output_variables = ['restaraunt_name','menu_items']
)



/tmp/ipython-input-3-994473038.py:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature = 0.9)
/tmp/ipython-input-3-994473038.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  name_chain=LLMChain(llm=llm,prompt=prompt_template_name,output_key="restaraunt_name")


In [4]:
chain({"cuisine": "pastries"})

/tmp/ipython-input-4-4276649775.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"cuisine": "pastries"})


{'cuisine': 'pastries',
 'restaraunt_name': '.\n\n"Sweet Indulgence Café"',
 'menu_items': '\n\n1. Decadent chocolate truffle cake\n2. Warm apple cinnamon crumble\n3. Creamy New York style cheesecake\n4. Freshly baked croissants with assorted jams and spreads\n5. Fluffy buttermilk pancakes with maple syrup and whipped cream\n6. Classic French toast with a side of crisp bacon\n7. Fruit and cream waffles with a drizzle of honey\n8. Assorted macarons in flavors like raspberry, chocolate, and pistachio \n9. Warm chocolate chip cookies with a glass of milk \n10. Rich and creamy hot chocolate with marshmallows \n11. Gourmet sandwiches like chicken and avocado or grilled cheese with truffle oil \n12. Freshly brewed coffee and espresso drinks \n13. Seasonal fruit and yogurt parfaits \n14. Avocado toast topped with poached eggs and sprinkled with feta cheese \n15. Spinach and feta quiche \n16. Vegetable frittata with roasted potatoes \n17. Deli style bagels with smoked salmon and cream cheese \

In [3]:
pip install gradio

In [4]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import gradio as gr

# Load your LLM (make sure your OpenAI key is set via environment variable)
llm = OpenAI(temperature=0.9)

# Prompt to generate restaurant name
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a high-end restaurant serving {cuisine} cuisine. Suggest a unique, elegant, and brandable restaurant name."
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaraunt_name")

# Prompt to generate menu items
prompt_template_items = PromptTemplate(
    input_variables=['restaraunt_name', 'cuisine'],
    template=(
        "Based on the restaurant name '{restaraunt_name}', which serves {cuisine} cuisine, "
        "suggest a list of creative and appealing menu items. "
        "Organize them under categories: Drinks, Soups, Starters, Main Course, and Desserts. "
        "Only list item names under each category."
    )
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

# Sequential chain: Name → Menu
chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=['cuisine'],
    output_variables=['restaraunt_name', 'menu_items'],
    verbose=False
)

# Main function for Gradio
def generate_restaurant(cuisine_dropdown, cuisine_text):
    cuisine = cuisine_text if cuisine_text else cuisine_dropdown
    result = chain({'cuisine': cuisine})
    output = f"🍽️ **Restaurant Name:**\n{result['restaraunt_name']}\n\n"
    output += "**🍴 Welcome to [YourAppName] – The AI Restaurant Creator! 🍴**\n\n"
    output += f"📋 **Menu Items:**\n{result['menu_items']}"
    return output

# UI elements
cuisine_options = ["Italian", "Indian", "Chinese", "Mexican", "Japanese", "French", "Thai", "Korean", "Arabian"]

with gr.Blocks() as demo:
    gr.Markdown("## 🍱 Welcome to **Restaurant Name & Menu Generator**")

    with gr.Row():
        cuisine_dropdown = gr.Dropdown(label="Choose a Cuisine", choices=cuisine_options, value="Italian")
        cuisine_text = gr.Textbox(label="Or type your own cuisine (optional)", placeholder="e.g., Bengali, Ethiopian...")

    generate_btn = gr.Button("Generate Restaurant Name & Menu")
    output_box = gr.Markdown()

    generate_btn.click(fn=generate_restaurant, inputs=[cuisine_dropdown, cuisine_text], outputs=output_box)

demo.launch()


/tmp/ipython-input-4-4161212109.py:7: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.9)
/tmp/ipython-input-4-4161212109.py:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaraunt_name")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aea2c7ddcfd717bb3b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import gradio as gr

# Load LLM
llm = OpenAI(temperature=0.9)

# Prompt templates
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a high-end restaurant serving {cuisine} cuisine. Suggest a unique, elegant, and brandable restaurant name."
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaraunt_name")

prompt_template_items = PromptTemplate(
    input_variables=['restaraunt_name', 'cuisine'],
    template=(
        "Based on the restaurant name '{restaraunt_name}', which serves {cuisine} cuisine, "
        "suggest a list of creative and appealing menu items. "
        "Organize them under categories: Drinks, Soups, Starters, Main Course, and Desserts. "
        "Only list item names under each category."
    )
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

# Combine into Sequential Chain
chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=['cuisine'],
    output_variables=['restaraunt_name', 'menu_items'],
    verbose=False
)

# Main logic
def generate_restaurant(cuisine_dropdown, cuisine_text):
    cuisine = cuisine_text if cuisine_text else cuisine_dropdown
    if not cuisine:
        return "⚠️ Please choose or enter a cuisine."
    result = chain({'cuisine': cuisine})
    output = f"""
### 🍽️ **Restaurant Name:**
**{result['restaraunt_name'].strip('"')}**

---

### 🌟 Welcome to **🥢 TasteCrafter AI 🍴** – Your Smart Restaurant Planner!

---

### 📋 **Menu Items by Category:**
{result['menu_items']}
"""
    return output

def clear_output():
    return "", ""

# Cuisine choices
cuisine_options = ["Italian", "Indian", "Chinese", "Mexican", "Japanese", "French", "Thai", "Korean", "Arabian"]

# Gradio UI
with gr.Blocks(css=".gr-button {font-weight: bold;}") as demo:
    gr.Markdown("## 🧑‍🍳 **Create Your Dream Restaurant in Seconds!**")
    gr.Markdown("#### ✨ Powered by LangChain + GPT ✨")

    with gr.Row():
        cuisine_dropdown = gr.Dropdown(label="Choose a Cuisine", choices=cuisine_options, value=None, scale=1)
        cuisine_text = gr.Textbox(label="Or type your own cuisine", placeholder="e.g., Bengali, Ethiopian...", scale=1)

    with gr.Row():
        generate_btn = gr.Button("🚀 Generate")
        clear_btn = gr.Button("🧼 Clear")

    output_box = gr.Markdown()

    generate_btn.click(fn=generate_restaurant, inputs=[cuisine_dropdown, cuisine_text], outputs=output_box)
    clear_btn.click(fn=clear_output, inputs=[], outputs=[output_box, cuisine_text])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d72b176afef443628.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Agents

In [6]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key
#get your free serp api
os.environ['SERPAPI_API_KEY'] = "c5fe69a76a9edc97f9f0d53d4ce913cae4b40216ae918321c05f0924b24c3429"

In [17]:
import os
from google.colab import userdata

SERPAPI_API_KEY = userdata.get("SERPAPI_API_KEY")

# Set them as environment variables

if SERPAPI_API_KEY:
    os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

SecretNotFoundError: Secret SERPAPI_API_KEY does not exist.

In [7]:
pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=800256fe0c6725c9f483351fdb6703c5f0fc0773e22704da369137171586472e
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


#### serpapi and llm-math tool

In [10]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
import os

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi","wikipedia", "llm-math"], llm=llm, serpapi_api_key=os.environ.get("SERPAPI_API_KEY"))

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What is [the numerical GDP value] + 5?")



> Entering new AgentExecutor chain...
 I should use the Calculator tool to solve this math problem.
Action: Calculator
Action Input: [the numerical GDP value] + 5

ValueError: LLMMathChain._evaluate("
[numerical GDP value] + 5
") raised error: Expression [numerical GDP value] + 5 has forbidden control characters.. Please try again with a valid numerical expression

#### Wikipedia and llm-math tool

In [8]:
pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=f3421935bb0b8f9b7da54e87d7e37a807bfcc5879607e1f3aeb0647e99364f0e
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [11]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Let's test it out!
#agent.run("When was Elon musk born? What is his surrent age")
agent.run("How many balloons can we fit in a380?")





> Entering new AgentExecutor chain...
 We need to find the volume of the a380 and the volume of a balloon to determine how many can fit.
Action: Calculator
Action Input: Volume of a380 = 845 m^3, Volume of a balloon = 0.014 m^3
Observation: Answer: 845.014
Thought: This is the total volume of the a380 and the balloon combined.
Action: Calculator
Action Input: Divide 845.014 by 0.014
Observation: Answer: 60358.142857142855
Thought: This is the maximum number of balloons that can fit in the a380.
Final Answer: 60358 balloons can fit in a380.

> Finished chain.


'60358 balloons can fit in a380.'

In [24]:
pip install --upgrade langchain langchain-community


In [25]:
pip install wikipedia serpapi openai gradio


In [14]:
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 1.9 MB/s eta 0:00:00


In [26]:
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_openai import OpenAI
# Corrected import path for PythonREPLTool
from langchain_community.tools.python.tool import PythonREPLTool
import gradio as gr
from google.colab import userdata

# 🔐 Get API keys from Colab Secrets
openai_api_key = userdata.get("openai")
serpapi_api_key = userdata.get("serpapi")

# ✅ Use OpenAI LLM
llm = OpenAI(temperature=0, api_key=openai_api_key)

# ✅ Load standard tools
tools = load_tools(
    ["wikipedia", "serpapi", "llm-math"],
    llm=llm,
    serpapi_api_key=serpapi_api_key
)

# ✅ Add Python REPL tool manually (no longer in string list)
tools.append(PythonREPLTool())

# ✅ Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# 🧠 Agent function
def ask_agent(question):
    try:
        return agent.run(question)
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# 🎨 Gradio Interface
with gr.Blocks(css=".gr-button {font-weight: bold;}") as demo:
    gr.Markdown("## 🤖 **Smart AI Research Agent**")
    gr.Markdown(
        "Ask anything! This agent can:\n"
        "- 🔍 Search Wikipedia\n"
        "- 🔢 Do math\n"
        "- 🧮 Run Python code\n"
        "- 🌐 Use live web search via SerpAPI"
    )

    question = gr.Textbox(label="Your Question", placeholder="e.g. Who founded Ethereum? What is 2^100?")
    submit = gr.Button("🚀 Ask Agent")
    clear = gr.Button("🧹 Clear")

    output = gr.Textbox(label="Agent Response", lines=12)

    submit.click(fn=ask_agent, inputs=question, outputs=output)
    clear.click(fn=lambda: "", inputs=[], outputs=output)

demo.launch()

ModuleNotFoundError: No module named 'langchain_community.tools.python'

In [28]:


from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_openai import OpenAI
import gradio as gr
from google.colab import userdata

# 🔐 API keys
openai_api_key = userdata.get("openai")
serpapi_api_key = userdata.get("serpapi")

# ✅ Load LLM
llm = OpenAI(temperature=0, api_key=openai_api_key)

# ✅ Load tools (no python tool!)
tools = load_tools(
    ["wikipedia", "serpapi", "llm-math", "pubmed", "ddg-search"],
    llm=llm,
    serpapi_api_key=serpapi_api_key
)

# ✅ Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# 🔁 Agent logic
def ask_agent(question):
    try:
        if any(x in question.lower() for x in ["calculate", "%", "+", "-", "*", "/", "what is", "how much"]):
            return agent.run(question)
        else:
            # Optional: append helpful instruction
            question = f"{question}. Use search or Wikipedia if helpful."
            return agent.run(question)
    except Exception as e:
        return f"⚠️ Error: {str(e)}"


# 🎨 Gradio interface
with gr.Blocks(css=".gr-button {font-weight: bold;}") as demo:
    gr.Markdown("## 🤖 **Smart AI Research Agent**")
    gr.Markdown(
        "This agent can:\n"
        "- 🔍 Search Wikipedia\n"
        "- 🌐 Perform real-time web search (SerpAPI & DuckDuckGo)\n"
        "- 📊 Solve math problems\n"
        "- 🧬 Search scientific research (PubMed)"
    )

    question = gr.Textbox(label="Your Question", placeholder="e.g. Latest Nobel Prize winner? What is 25% of 1.2M?")
    submit = gr.Button("🚀 Ask Agent")
    clear = gr.Button("🧹 Clear")

    output = gr.Textbox(label="Agent Response", lines=12)

    submit.click(fn=ask_agent, inputs=question, outputs=output)
    clear.click(fn=lambda: "", inputs=[], outputs=output)

demo.launch()


<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=85, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=86, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=85, family=2, type=1, proto=0

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e029eb95561a60f0a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [23]:
!pip install xmltodict

In [25]:
!pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.2 MB/s eta 0:00:00


In [30]:
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_openai import OpenAI
import gradio as gr
from google.colab import userdata

# 🔐 API keys
openai_api_key = userdata.get("openai")
serpapi_api_key = userdata.get("serpapi")

# ✅ Load LLM
llm = OpenAI(temperature=0, api_key=openai_api_key)

# ✅ Load reliable tools
tools = load_tools(
    ["wikipedia", "serpapi", "ddg-search", "pubmed"],
    llm=llm,
    serpapi_api_key=serpapi_api_key
)

# ✅ Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# 🧠 Agent function
def ask_agent(question):
    try:
        return agent.run(question)
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# 🎨 Gradio UI
with gr.Blocks(css=".gr-button {font-weight: bold;}") as demo:
    gr.Markdown("## 🤖 **Smart AI Research Agent**")
    gr.Markdown(
        "This agent can:\n"
        "- 🔍 Search Wikipedia\n"
        "- 🌐 Perform web search (SerpAPI + DuckDuckGo)\n"
        "- 🧬 Find scientific papers (PubMed)"
    )

    question = gr.Textbox(label="Your Question", placeholder="e.g. What is the role of serotonin?")
    submit = gr.Button("🚀 Ask Agent")
    clear = gr.Button("🧹 Clear")

    output = gr.Textbox(label="Agent Response", lines=12)

    submit.click(fn=ask_agent, inputs=question, outputs=output)
    clear.click(fn=lambda: "", inputs=[], outputs=output)

demo.launch()


/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=96, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=97, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=96, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=97, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://62340762bad9d0d9d7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


IMPROVISED UI

In [31]:
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_openai import OpenAI
import gradio as gr
from google.colab import userdata

# 🔐 API keys
openai_api_key = userdata.get("openai")
serpapi_api_key = userdata.get("serpapi")

# ✅ Load LLM
llm = OpenAI(temperature=0, api_key=openai_api_key)

# ✅ Load tools
tools = load_tools(
    ["wikipedia", "serpapi", "ddg-search", "pubmed"],
    llm=llm,
    serpapi_api_key=serpapi_api_key
)

# ✅ Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# 🧠 Agent function
def ask_agent(question):
    try:
        return agent.run(question)
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# 🧹 Clear only input
def clear_input():
    return ""

# 🎨 Gradio UI
with gr.Blocks(css="""
    .main-title {text-align: center; font-size: 30px; font-weight: bold; margin-bottom: 10px;}
    .gr-button {font-weight: bold;}
    .gr-textbox {border-radius: 10px;}
    textarea {font-family: monospace;}
""") as demo:

    gr.Markdown("<div class='main-title'>🤖 Smart AI Research Agent</div>")

    gr.Markdown(
        "This agent can:\n"
        "- 🔍 Search Wikipedia\n"
        "- 🌐 Perform web search (SerpAPI + DuckDuckGo)\n"
        "- 🧬 Find scientific papers (PubMed)\n\n"
        "Type your question below and click **Ask Agent**."
    )

    with gr.Row():
        with gr.Column(scale=3):
            question = gr.Textbox(label="Your Question", placeholder="e.g. What is the role of serotonin?")
            btn_row = gr.Row()
            with btn_row:
                submit = gr.Button("🚀 Ask Agent")
                clear = gr.Button("🧹 Clear")
        with gr.Column(scale=5):
            output = gr.Textbox(label="Agent Response", lines=15, interactive=False)

    submit.click(fn=ask_agent, inputs=question, outputs=output)
    clear.click(fn=clear_input, inputs=[], outputs=question)

demo.launch()


/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=66, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=76, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=66, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=76, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=66, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://69ff410b72d1793ce6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
